<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/main/Feature_Engineering_from_Raw_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
import sys
from numpy.ma.core import ceil, floor
from more_itertools import sliced

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/

210815_0__20220213T070259RAW_stimulus_data.csv
210815_0_allStacks_mc_mix1_syt_rawExtracts


## Functions
This section contains functions that can be used on datasets for feature engineering

In [31]:
raw_df = pd.read_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0__20220213T070259RAW_stimulus_data.csv', index_col=['stack','label','roi','frame']).drop(columns='Unnamed: 0')
raw_df

filename  ...     stim3
stack label roi frame                                                     ...          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.110063
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.030790
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.122134
...                                                                  ...  ...       ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.089182
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.019588
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000

[826950 rows x 5 columns]

## Train Test Split on Stacks

In [43]:
def trainTestSplit(df, splt = [.7,.2,.1]):
  # splits on stack
  # split is the train test valuidation split (proportions given in that order)
  if sum(splt) < 0.9999 or sum(splt) > 1.0001:
    sys.exit('Splt must add to 1')
  
  num_stacks = raw_df.index.unique(level='stack').max()

  tr_ind = int(ceil(num_stacks*splt[0]))
  ts_ind = int(ceil(num_stacks*splt[1])) + tr_ind

  train = ar[:tr_ind]
  test = ar[tr_ind:ts_ind]
  val = ar[ts_ind:]

  for i in train:
    df.loc[i,'training'] = 'train'

  for i in test:
    df.loc[i,'training'] = 'test'

  for i in val:
    df.loc[i,'training'] = 'validate'

  return(df)

In [46]:
# df_temp = trainTestSplit(raw_df,splt=[.5,.3,.3]) # should give error
df_temp = trainTestSplit(raw_df,splt=[.5,.3,.2])
df_temp.sample(15)

/usr/local/lib/python3.7/dist-packages/ipykernel/ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


,,,,filename,resp,stim1,stim2,stim3,training
stack,label,roi,frame,,,,,,
4,tm9,6,4780,210815_0_4_stackRaw_mc_mix1_syt_result_2022021...,0.114053,-0.891936,-78.674800,0.049869,train
1,t5,12,334,210815_0_1_stackRaw_mc_mix1_syt_result_2022021...,0.293874,0.380138,0.000000,0.000000,validate
5,tm9,6,3160,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,-0.391502,0.089696,104.532567,0.000023,test
4,tm4,3,4399,210815_0_4_stackRaw_mc_mix1_syt_result_2022021...,0.040225,-0.132008,0.000000,0.000000,train
3,tm1,7,10,210815_0_3_stackRaw_mc_mix1_syt_result_2022021...,0.804364,-0.417031,0.000000,0.000000,test
2,tm2,5,1919,210815_0_2_stackRaw_mc_mix1_syt_result_2022021...,-0.005911,0.134887,127.783822,0.042764,validate
1,t5,12,4492,210815_0_1_stackRaw_mc_mix1_syt_result_2022021...,0.261175,-0.158865,123.274978,0.005118,validate
5,tm2,10,5398,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,-0.395127,0.270286,-52.899402,0.007218,test
4,tm2,1,4598,210815_0_4_stackRaw_mc_mix1_syt_result_2022021...,-0.172616,-0.039544,-56.957473,0.010234,train


## Combine Frames into Temporal Chunks

In [ ]:
# select frame size
temporal_period_length = 200

In [109]:

def groupFrames(df, period):
  # inputs are data frame and temporal period length
  # df needs to be sorted correctly, the indexing should take care of that

  # check that all rois are the same size
  # if this is a problem we can write a more computationally heavy workthrough
  if (raw_df.groupby(['stack','roi']).count()['filename'].max() != raw_df.groupby(['stack','roi']).count()['filename'].min()):
    sys.exit('ROIs need to be the same length')

  num_stacks = raw_df.index.unique(level='stack').max()
  num_roi = raw_df.index.unique(level='roi').max()

  # get first label
  lbl_1 = raw_df.head(1).index.values[0][1]
  # use it to find the number of frames in a roi
  num_frames = raw_df.loc[(1,lbl_1,roi),'filename'].count()
  # how many groups will ther be in each roi?
  num_pds = int(floor(num_frames/period))
  # how many left over
  remainder_pds = num_frames%period

  # iterate a column
  lst = [[li + 1] * period for li in range(0,num_pds)]
  lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
  lst = lst + [lst[-1] + 1] * remainder_pds
  lst

  new_col = np.array(lst * num_stacks * num_roi)
  
  df['frame_group'] = new_col

  return(df)

In [110]:
df_temp2 = groupFrames(raw_df,200)
df_temp2

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


filename  ...  frame_group
stack label roi frame                                                     ...             
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
...                                                                  ...  ...          ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28

[826950 rows x 7 columns]

In [68]:
10001%10

1